# Transfer Learning

Researchers around the world compete with each other to build the most accurate and capable image recognition systems. So instead of them bending their own neural network designs from scratch, it often makes sense to reuse an existing neural network design as a starting point for your own projects. But even better, researchers also trained these neural network designs on large data sets and share the trained versions of the neural networks. So we can take those pre trained neural networks and either reuse them directly, or use them as a starting point for our own training. 

Keras the library includes copies of many popular pre trained neural networks that are ready to use. The image recognition models included with Keras are all trained to recognize images from the ImageNet data set. The ImageNet data set is a collection of millions of pictures of objects that have been labeled so that you can use them to train computers to recognize those objects. The date set includes over 1200 pictures of just this specific kind of apple. Let's talk about the neural network designs included with Kares that we can reuse. 

First is **VGG**, VGG is a deep neural network with either 16 or 19 layers. It was the state of the art in 2014. It's a very standard convolutional neural network design. It's still used widely today as a basis for other models because it's easy to work with and easy to understand. But newer designs tend to be more efficient. **ResNet-50** is a state of the art from 2015, it's a 50 layer neural network that manages to be more accurate, but use less memory than the VGG design. ResNet uses a more complicated design, where higher layers in the neural network are connected not just the layer directly below them, but they also have multiple connections to deeper layers. **Inception v3** is another design from 2015 that also performs very well. It has an even more complex design based around layers that branch out into multiple separate paths before rejoining. 

These networks show the research trends in 2014 and 2015 to make neural networks bigger and more complex that try to increase their accuracy. More recent neural network designs tend to be more specialized. For example, **Google's MobileNet** created in 2017 is designed specifically to be able to run well on low power devices. The idea was to create a neural network that could run quickly on a cell phone without using too much power while still maintaining a decent level of accuracy. **Google's NASNet** which was created at the end of 2017, explores the idea of having algorithms design neural networks. 

Having access to these pre trained models is useful for two reasons. First you can reuse any of these models directly in your own programs to recognize objects and images. If you need the ability to recognize any of the 1,000 types of object they're already trained on, you're problems already solved. Second if you want to recognize a different type of object that's not in the 1,000 object training set, it's much faster to start with a pre trained neural network and adapt it to your needs, instead of training your own model from scratch. 

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

----
## Import libraries
All the pretrained models included with Keras are under the applications package. Import the model by saying from Keras applications import vgg16.

In [1]:
import numpy as np
from keras.preprocessing import image
from keras.applications import vgg16

Using TensorFlow backend.


----
## Image recognition pretrained (VGG16)
The first time we run this code, Keras will connect to the internet and download the latest version of the vgg16 model. This means that you'll need internet access to run it, and around 100 megabytes of data will be downloaded. 

In [2]:
# Load Keras' VGG16 model that was pre-trained against the ImageNet database
model = vgg16.VGG16()

# Load the image file, resizing it to 224x224 pixels (required by VGG model)
img = image.load_img("bay.jpg", target_size=(224, 224))

# Convert the image to a numpy array
x = image.img_to_array(img)

# Add a fourth dimension (since Keras expects a list of images)
x = np.expand_dims(x, axis=0)

# Normalize the input image's pixel values to the range used when training the neural network
x = vgg16.preprocess_input(x)

# Run the image through the deep neural network to make a prediction
predictions = model.predict(x)

# Look up the names of the predicted classes. Index zero is the results for the first image.
predicted_classes = vgg16.decode_predictions(predictions, top=9)

print("Top predictions for this image:")

for imagenet_id, name, likelihood in predicted_classes[0]:
    print("Prediction: {} - {:2f}".format(name, likelihood))

Instructions for updating:
Colocations handled automatically by placer.
Top predictions for this image:
Prediction: seashore - 0.395212
Prediction: promontory - 0.326130
Prediction: lakeside - 0.119613
Prediction: breakwater - 0.062801
Prediction: sandbar - 0.045267
Prediction: cliff - 0.011845
Prediction: dock - 0.009196
Prediction: boathouse - 0.003278
Prediction: valley - 0.003194


----
## Transfer learning
We can use transfer learning to reuse an existing neural network and adapt it to a new problem. Transfer learning is where you take a model trained on one set of data and then use the knowledge it learned to give it a headstart when solving a new problem. 

The first step is to build a feature extractor that can extract training features from our images. 

In [3]:
from pathlib import Path
import joblib

In [4]:
# Path to folders with training data
dog_path = Path("training_data") / "dogs"
not_dog_path = Path("training_data") / "not_dogs"

images = []
labels = []

# Load all the not-dog images
for img in not_dog_path.glob("*.png"):
    # Load the image from disk
    img = image.load_img(img)

    # Convert the image to a numpy array
    image_array = image.img_to_array(img)

    # Add the image to the list of images
    images.append(image_array)

    # For each 'not dog' image, the expected value should be 0
    labels.append(0)

# Load all the dog images
for img in dog_path.glob("*.png"):
    # Load the image from disk
    img = image.load_img(img)

    # Convert the image to a numpy array
    image_array = image.img_to_array(img)

    # Add the image to the list of images
    images.append(image_array)

    # For each 'dog' image, the expected value should be 1
    labels.append(1)

# Create a single numpy array with all the images we loaded
x_train = np.array(images)

# Also convert the labels to a numpy array
y_train = np.array(labels)

# Normalize image data to 0-to-1 range
x_train = vgg16.preprocess_input(x_train)

# Load a pre-trained neural network to use as a feature extractor
pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Extract features for each image (all in one pass)
features_x = pretrained_nn.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, "x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "y_train.dat")


58892288/58889256 [==============================] - 5s 0us/step


['y_train.dat']

We've used the pre trained neural network to extract features from our training images. Now we're ready to train a new neural network that uses those extracted features. Instead of loading raw images to train with, we're gonna load the features that we extracted with the pre trained VGG 16 neural network. Since we use VGG 16 to extract features from our image, this neural network has no convolutional layers. Instead it only has the final dense layers of the neural network. These are the only layers that we'll be retraining. 

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [6]:
# Load data set
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")

# Create a model and add layers
model = Sequential()

model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)

# Train the model
model.fit(
    x_train,
    y_train,
    epochs=10,
    shuffle=True
)

# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model_weights.h5")


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
58/58 [==============================] - 3s 55ms/step - loss: 4.5036 - acc: 0.6034
Epoch 2/10
58/58 [==============================] - 0s 902us/step - loss: 1.8010 - acc: 0.8621
Epoch 3/10
58/58 [==============================] - 0s 971us/step - loss: 1.2681 - acc: 0.8966
Epoch 4/10
58/58 [==============================] - 0s 1ms/step - loss: 0.8246 - acc: 0.9483
Epoch 5/10
58/58 [==============================] - 0s 1ms/step - loss: 0.9564 - acc: 0.9310
Epoch 6/10
58/58 [==============================] - 0s 1ms/step - loss: 0.8246 - acc: 0.9483
Epoch 7/10
58/58 [==============================] - 0s 1ms/step - loss: 0.8326 - acc: 0.9483
Epoch 8/10
58/58 [==============================] - 0s 1ms/step - loss: 0.8246 - acc: 0.9483
Epoch 9/10
58/58 [==============================] - 0s 792us/step - loss: 0.8246 - acc: 

We've used transfer learning to create and train a neural network that can recognize pictures of dogs. Let's see how they use that neural network to make predictions. We'll need the pre processor image with the vgg16 feature extractor.

In [7]:
from keras.models import model_from_json

In [8]:
# Load the json file that contains the model's structure
f = Path("model_structure.json")
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

# Re-load the model's trained weights
model.load_weights("model_weights.h5")

# Load an image file to test, resizing it to 64x64 pixels (as required by this model)
img = image.load_img("not_dog.png", target_size=(64, 64))

# Convert the image to a numpy array
image_array = image.img_to_array(img)

# Add a forth dimension to the image (since Keras expects a bunch of images, not a single image)
images = np.expand_dims(image_array, axis=0)

# Normalize the data
images = vgg16.preprocess_input(images)

# Use the pre-trained neural network to extract features from our test image (the same way we did to train the model)
feature_extraction_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
features = feature_extraction_model.predict(images)

# Given the extracted features, make a final prediction using our own model
results = model.predict(features)

# Since we are only testing one image with possible class, we only need to check the first result's first element
single_result = results[0][0]

# Print the result
print("Likelihood that this image contains a dog: {}%".format(int(single_result * 100)))

Likelihood that this image contains a dog: 0%


In [9]:
img = image.load_img("dog.png", target_size=(64, 64))
image_array = image.img_to_array(img)
images = np.expand_dims(image_array, axis=0)
images = vgg16.preprocess_input(images)
feature_extraction_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
features = feature_extraction_model.predict(images)
results = model.predict(features)
single_result = results[0][0]
print("Likelihood that this image contains a dog: {}%".format(int(single_result * 100)))

Likelihood that this image contains a dog: 100%


In [11]:
img = image.load_img("bay.jpg", target_size=(64, 64))
image_array = image.img_to_array(img)
images = np.expand_dims(image_array, axis=0)
images = vgg16.preprocess_input(images)
feature_extraction_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
features = feature_extraction_model.predict(images)
results = model.predict(features)
single_result = results[0][0]
print("Likelihood that this image contains a dog: {}%".format(int(single_result * 100)))

Likelihood that this image contains a dog: 0%
